In [24]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS
import SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
#wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
artchives_endpoint = "http://artchives.fondazionezeri.unibo.it/sparql"

# bind the uncommon namespaces
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")

# create an empty Graph
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("artchives.nq", format='nquads')


query_result = g.query(
    """PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT (SAMPLE(?coll_label) AS ?coll) (SAMPLE(?label) AS ?period_label) ?earliest ?latest
    WHERE {?coll <https://w3id.org/artchives/hasSubjectPeriod> ?period ; rdfs:label ?coll_label .
    ?period rdfs:label ?label .
    OPTIONAL {?coll wdt:P1319 ?earliest}
    OPTIONAL {?coll wdt:P1326 ?latest}
    }
    GROUP BY ?period ?coll ?earliest ?latest
    ORDER BY ?period_label""")



In [46]:
# the new list
periods_dates_expanded = []

# query the list of tuples of query results
for res in query_result:
    if res["earliest"] is not None and res["latest"] is not None:
        timespan = list(range( int(res["earliest"][:4]), int(res["latest"][:4])+1 ))
        for year in timespan:
            periods_dates_expanded.append( (res["coll"], res["period_label"], str(year)) ) # append a tuple!

In [49]:
# prepare the csv file
import csv
import pandas as pd

with open('periods_dates.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['collection', 'period_label', 'year'])
    
    # access the rows of the query results
    for coll, label, year in periods_dates_expanded:
        # write in the csv
        my_writer.writerow([coll.strip(), label.strip(), year])

In [50]:
# parse the csv into a dataframe
data = pd.read_csv("periods_dates.csv")
# print the first 59 rows
data.head(60)

,collection,period_label,year
0,Fototeca Zeri,Baroque,1947
1,Fototeca Zeri,Baroque,1948
2,Fototeca Zeri,Baroque,1949
3,Fototeca Zeri,Baroque,1950
4,Fototeca Zeri,Baroque,1951
5,Fototeca Zeri,Baroque,1952
6,Fototeca Zeri,Baroque,1953
7,Fototeca Zeri,Baroque,1954
8,Fototeca Zeri,Baroque,1955
9,Fototeca Zeri,Baroque,1956


In [ ]:
#trovare un modo per trovare i due anni, uno earliest e l'altro latest